# Netflix Show Clustering Analysis

This notebook demonstrates K-Means clustering to group similar Netflix shows based on genre, rating, and duration.

## Project Overview
- **Objective**: Group similar Netflix shows using unsupervised machine learning
- **Algorithm**: K-Means Clustering
- **Features**: Genre, Rating, Duration, Type, Year, Seasons
- **Tech Stack**: Python, Pandas, Scikit-learn, Seaborn, Matplotlib

## Dataset
We'll analyze a curated dataset of Netflix shows and movies with features including:
- **Title**: Name of the show/movie
- **Genre**: Content category (Action, Drama, Comedy, etc.)
- **Rating**: IMDb rating (1-10 scale)
- **Duration**: Length in minutes
- **Type**: Movie or TV Show
- **Year**: Release year
- **Seasons**: Number of seasons (for TV shows)

## 1. Import Required Libraries

In [ ]:
# Import required libraries for data manipulation, clustering, and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning libraries
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")
print("📚 Libraries loaded:")
print("   - Pandas for data manipulation")
print("   - NumPy for numerical operations") 
print("   - Matplotlib & Seaborn for visualization")
print("   - Scikit-learn for machine learning")

## 2. Load and Explore Netflix Dataset

In [ ]:
# Load the Netflix dataset
df = pd.read_csv('netflix_shows.csv')

print("📊 Dataset Overview:")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print("\n📋 Data Types:")
print(df.dtypes)
print("\n🔍 Missing Values:")
print(df.isnull().sum())
print("\n📈 Basic Statistics:")
print(df.describe())

In [ ]:
# Display first few rows of the dataset
print("🎬 Sample Netflix Shows:")
print("="*80)
df.head(10)

In [ ]:
# Exploratory Data Analysis - Visualizations
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('🎬 Netflix Shows - Exploratory Data Analysis', fontsize=16, fontweight='bold')

# 1. Rating distribution
axes[0, 0].hist(df['rating'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Rating Distribution')
axes[0, 0].set_xlabel('Rating')
axes[0, 0].set_ylabel('Count')
axes[0, 0].grid(True, alpha=0.3)

# 2. Duration distribution
axes[0, 1].hist(df['duration'], bins=20, alpha=0.7, color='lightgreen', edgecolor='black')
axes[0, 1].set_title('Duration Distribution')
axes[0, 1].set_xlabel('Duration (minutes)')
axes[0, 1].set_ylabel('Count')
axes[0, 1].grid(True, alpha=0.3)

# 3. Genre distribution
genre_counts = df['genre'].value_counts()
axes[0, 2].bar(range(len(genre_counts)), genre_counts.values, color='coral', alpha=0.8)
axes[0, 2].set_title('Genre Distribution')
axes[0, 2].set_xlabel('Genre')
axes[0, 2].set_ylabel('Count')
axes[0, 2].set_xticks(range(len(genre_counts)))
axes[0, 2].set_xticklabels(genre_counts.index, rotation=45, ha='right')

# 4. Rating vs Duration scatter plot
colors = ['red' if t == 'Movie' else 'blue' for t in df['type']]
axes[1, 0].scatter(df['duration'], df['rating'], c=colors, alpha=0.7, s=50)
axes[1, 0].set_title('Rating vs Duration')
axes[1, 0].set_xlabel('Duration (minutes)')
axes[1, 0].set_ylabel('Rating')
axes[1, 0].grid(True, alpha=0.3)
# Create custom legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='red', label='Movie'),
                  Patch(facecolor='blue', label='TV Show')]
axes[1, 0].legend(handles=legend_elements)

# 5. Year distribution
axes[1, 1].hist(df['year'], bins=15, alpha=0.7, color='gold', edgecolor='black')
axes[1, 1].set_title('Release Year Distribution')
axes[1, 1].set_xlabel('Year')
axes[1, 1].set_ylabel('Count')
axes[1, 1].grid(True, alpha=0.3)

# 6. Type distribution
type_counts = df['type'].value_counts()
axes[1, 2].pie(type_counts.values, labels=type_counts.index, autopct='%1.1f%%', 
               colors=['lightcoral', 'lightblue'])
axes[1, 2].set_title('Content Type Distribution')

plt.tight_layout()
plt.show()

# Print genre and type distributions
print("\n📊 Content Distribution:")
print("Genre counts:")
for genre, count in df['genre'].value_counts().items():
    print(f"  {genre}: {count}")

print(f"\nType counts:")
for content_type, count in df['type'].value_counts().items():
    print(f"  {content_type}: {count}")

## 3. Data Preprocessing and Feature Engineering

In [ ]:
# Create a copy for preprocessing
df_processed = df.copy()

# Initialize label encoders
label_encoders = {}

# Encode categorical variables
categorical_cols = ['genre', 'type', 'country']

print("🔄 Encoding categorical variables:")
for col in categorical_cols:
    if col in df_processed.columns:
        le = LabelEncoder()
        df_processed[f'{col}_encoded'] = le.fit_transform(df_processed[col])
        label_encoders[col] = le
        
        # Create mapping dictionary
        mapping = dict(zip(le.classes_, le.transform(le.classes_)))
        print(f"  {col}: {mapping}")

# Check for missing values and handle if necessary
print(f"\n🔍 Missing values after preprocessing:")
print(df_processed.isnull().sum())

# If there are missing values, fill them
if df_processed.isnull().sum().sum() > 0:
    print("⚠️ Handling missing values...")
    df_processed = df_processed.fillna(df_processed.mean(numeric_only=True))
    print("✅ Missing values handled")

print(f"\n✅ Preprocessing completed!")
print(f"Original features: {list(df.columns)}")
print(f"Processed features: {list(df_processed.columns)}")

## 4. Feature Scaling and Preparation

In [ ]:
# Select features for clustering
clustering_features = ['rating', 'duration', 'genre_encoded', 'type_encoded', 'year', 'seasons']

# Ensure all features exist in the dataframe
available_features = [f for f in clustering_features if f in df_processed.columns]
print(f"🎯 Selected features for clustering: {available_features}")

# Create feature matrix
X = df_processed[available_features].copy()

print(f"\n📊 Feature matrix before scaling:")
print(X.describe())

# Scale the features using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=available_features)

print(f"\n⚖️ Feature matrix after scaling:")
print(X_scaled_df.describe())

print(f"\n✅ Feature scaling completed!")
print(f"Feature matrix shape: {X_scaled_df.shape}")

# Visualize the scaled features
plt.figure(figsize=(12, 8))
plt.subplot(2, 1, 1)
X.boxplot()
plt.title('Features Before Scaling')
plt.xticks(rotation=45)

plt.subplot(2, 1, 2)
X_scaled_df.boxplot()
plt.title('Features After Scaling')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## 5. Determine Optimal Number of Clusters

In [ ]:
# Use Elbow Method and Silhouette Analysis to find optimal clusters
max_clusters = 10
inertias = []
silhouette_scores = []
K_range = range(2, max_clusters + 1)

print("🔍 Finding optimal number of clusters...")
print("K\tInertia\t\tSilhouette Score")
print("-" * 40)

for k in K_range:
    # Fit K-Means
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(X_scaled)
    
    # Calculate metrics
    inertia = kmeans.inertia_
    sil_score = silhouette_score(X_scaled, cluster_labels)
    
    inertias.append(inertia)
    silhouette_scores.append(sil_score)
    
    print(f"{k}\t{inertia:.2f}\t\t{sil_score:.3f}")

# Find optimal k using silhouette score
optimal_k = K_range[np.argmax(silhouette_scores)]
print(f"\n🎯 Optimal number of clusters: {optimal_k} (highest silhouette score)")

# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Elbow Method Plot
ax1.plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
ax1.set_xlabel('Number of Clusters (k)')
ax1.set_ylabel('Inertia (Within-cluster Sum of Squares)')
ax1.set_title('📈 Elbow Method for Optimal k')
ax1.grid(True, alpha=0.3)
ax1.axvline(x=optimal_k, color='red', linestyle='--', alpha=0.8,
           label=f'Optimal k = {optimal_k}')
ax1.legend()

# Silhouette Score Plot
ax2.plot(K_range, silhouette_scores, 'ro-', linewidth=2, markersize=8)
ax2.set_xlabel('Number of Clusters (k)')
ax2.set_ylabel('Silhouette Score')
ax2.set_title('📊 Silhouette Analysis')
ax2.grid(True, alpha=0.3)
ax2.axvline(x=optimal_k, color='red', linestyle='--', alpha=0.8,
           label=f'Optimal k = {optimal_k}')
ax2.legend()

plt.tight_layout()
plt.show()

print(f"\n📋 Analysis Summary:")
print(f"   • Optimal clusters: {optimal_k}")
print(f"   • Best silhouette score: {max(silhouette_scores):.3f}")
print(f"   • Corresponding inertia: {inertias[optimal_k-2]:.2f}")

## 6. Apply K-Means Clustering

In [ ]:
# Apply K-Means with optimal number of clusters
final_kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
cluster_labels = final_kmeans.fit_predict(X_scaled)

# Add cluster labels to the processed dataframe
df_processed['cluster'] = cluster_labels

# Calculate final metrics
final_inertia = final_kmeans.inertia_
final_silhouette = silhouette_score(X_scaled, cluster_labels)

print("🎯 K-Means Clustering Results:")
print(f"   • Number of clusters: {optimal_k}")
print(f"   • Final inertia: {final_inertia:.2f}")
print(f"   • Final silhouette score: {final_silhouette:.3f}")

# Display cluster distribution
cluster_distribution = df_processed['cluster'].value_counts().sort_index()
print(f"\n📊 Cluster Distribution:")
for cluster_id, count in cluster_distribution.items():
    percentage = (count / len(df_processed)) * 100
    print(f"   Cluster {cluster_id}: {count} shows ({percentage:.1f}%)")

# Create a summary table
print(f"\n📋 Cluster Summary:")
summary_stats = df_processed.groupby('cluster').agg({
    'rating': ['mean', 'std'],
    'duration': ['mean', 'std'],
    'year': ['min', 'max'],
    'seasons': 'mean'
}).round(2)

print(summary_stats)

## 7. Visualize Clustering Results

In [ ]:
# Create comprehensive visualizations
fig = plt.figure(figsize=(20, 15))

# 1. PCA visualization for 2D representation
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

ax1 = plt.subplot(2, 3, 1)
scatter = ax1.scatter(X_pca[:, 0], X_pca[:, 1], 
                     c=df_processed['cluster'], 
                     cmap='tab10', alpha=0.7, s=60)
ax1.set_title('🎯 Clusters in PCA Space')
ax1.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
ax1.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
ax1.grid(True, alpha=0.3)
plt.colorbar(scatter, ax=ax1, label='Cluster')

# 2. Rating vs Duration scatter plot
ax2 = plt.subplot(2, 3, 2)
scatter2 = ax2.scatter(df_processed['duration'], df_processed['rating'], 
                      c=df_processed['cluster'], cmap='tab10', alpha=0.7, s=60)
ax2.set_title('📊 Rating vs Duration (Clustered)')
ax2.set_xlabel('Duration (minutes)')
ax2.set_ylabel('Rating')
ax2.grid(True, alpha=0.3)
plt.colorbar(scatter2, ax=ax2, label='Cluster')

# 3. Cluster size distribution
ax3 = plt.subplot(2, 3, 3)
cluster_counts = df_processed['cluster'].value_counts().sort_index()
bars = ax3.bar(cluster_counts.index, cluster_counts.values, 
              color=plt.cm.tab10(np.linspace(0, 1, len(cluster_counts))), alpha=0.8)
ax3.set_title('📈 Cluster Size Distribution')
ax3.set_xlabel('Cluster')
ax3.set_ylabel('Number of Shows')
ax3.grid(True, alpha=0.3)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}', ha='center', va='bottom')

# 4. Average rating by cluster
ax4 = plt.subplot(2, 3, 4)
avg_rating = df_processed.groupby('cluster')['rating'].mean()
bars4 = ax4.bar(avg_rating.index, avg_rating.values,
               color=plt.cm.tab10(np.linspace(0, 1, len(avg_rating))), alpha=0.8)
ax4.set_title('⭐ Average Rating by Cluster')
ax4.set_xlabel('Cluster')
ax4.set_ylabel('Average Rating')
ax4.set_ylim(0, 10)
ax4.grid(True, alpha=0.3)

# Add value labels
for bar in bars4:
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.1f}', ha='center', va='bottom')

# 5. Genre distribution by cluster
ax5 = plt.subplot(2, 3, 5)
genre_cluster = pd.crosstab(df_processed['cluster'], df_processed['genre'])
genre_cluster.plot(kind='bar', stacked=True, ax=ax5, colormap='Set3')
ax5.set_title('🎭 Genre Distribution by Cluster')
ax5.set_xlabel('Cluster')
ax5.set_ylabel('Count')
ax5.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax5.grid(True, alpha=0.3)
plt.setp(ax5.xaxis.get_majorticklabels(), rotation=0)

# 6. Type distribution by cluster
ax6 = plt.subplot(2, 3, 6)
type_cluster = pd.crosstab(df_processed['cluster'], df_processed['type'])
type_cluster.plot(kind='bar', ax=ax6, colormap='Set2', alpha=0.8)
ax6.set_title('📺 Type Distribution by Cluster')
ax6.set_xlabel('Cluster')
ax6.set_ylabel('Count')
ax6.legend()
ax6.grid(True, alpha=0.3)
plt.setp(ax6.xaxis.get_majorticklabels(), rotation=0)

plt.tight_layout()
plt.show()

print("🎨 All visualizations completed!")

## 8. Analyze Cluster Characteristics

In [ ]:
# Detailed analysis of each cluster
n_clusters = len(df_processed['cluster'].unique())

print("🔍 DETAILED CLUSTER ANALYSIS")
print("=" * 60)

for cluster_id in sorted(df_processed['cluster'].unique()):
    cluster_data = df_processed[df_processed['cluster'] == cluster_id]
    
    print(f"\n🎯 CLUSTER {cluster_id} ANALYSIS")
    print(f"{'=' * 30}")
    print(f"📊 Size: {len(cluster_data)} shows ({len(cluster_data)/len(df_processed)*100:.1f}%)")
    
    # Statistical summary
    print(f"\n📈 Statistical Summary:")
    print(f"   • Average Rating: {cluster_data['rating'].mean():.2f} (±{cluster_data['rating'].std():.2f})")
    print(f"   • Average Duration: {cluster_data['duration'].mean():.1f} minutes (±{cluster_data['duration'].std():.1f})")
    print(f"   • Year Range: {cluster_data['year'].min()} - {cluster_data['year'].max()}")
    print(f"   • Average Seasons: {cluster_data['seasons'].mean():.1f}")
    
    # Most common characteristics
    print(f"\n🏷️ Most Common Characteristics:")
    most_common_genre = cluster_data['genre'].mode()
    most_common_type = cluster_data['type'].mode()
    most_common_country = cluster_data['country'].mode()
    
    print(f"   • Genre: {most_common_genre.iloc[0] if len(most_common_genre) > 0 else 'N/A'}")
    print(f"   • Type: {most_common_type.iloc[0] if len(most_common_type) > 0 else 'N/A'}")
    print(f"   • Country: {most_common_country.iloc[0] if len(most_common_country) > 0 else 'N/A'}")
    
    # Genre distribution in this cluster
    print(f"\n🎭 Genre Distribution:")
    genre_dist = cluster_data['genre'].value_counts()
    for genre, count in genre_dist.head().items():
        percentage = (count / len(cluster_data)) * 100
        print(f"   • {genre}: {count} shows ({percentage:.1f}%)")
    
    # Sample shows from this cluster
    print(f"\n🎬 Sample Shows:")
    sample_shows = cluster_data[['title', 'genre', 'rating', 'duration', 'type']].head(5)
    for _, show in sample_shows.iterrows():
        print(f"   • {show['title']} ({show['genre']}, {show['type']}) - ⭐{show['rating']}, {show['duration']}min")

print(f"\n" + "=" * 60)

In [ ]:
# Generate key insights and business recommendations
print("💡 KEY INSIGHTS & BUSINESS RECOMMENDATIONS")
print("=" * 60)

insights = []

# Clustering quality assessment
if final_silhouette > 0.5:
    insights.append("✅ Excellent clustering quality - clear separation between clusters")
elif final_silhouette > 0.3:
    insights.append("📊 Good clustering quality - meaningful groupings identified")
else:
    insights.append("⚠️ Moderate clustering quality - consider feature engineering")

# Cluster balance
max_size = cluster_distribution.max()
min_size = cluster_distribution.min()
if max_size / min_size <= 3:
    insights.append("⚖️ Well-balanced cluster sizes")
else:
    insights.append("📏 Some clusters are significantly larger than others")

# Content type separation
type_analysis = pd.crosstab(df_processed['cluster'], df_processed['type'])
for cluster_id in type_analysis.index:
    movies = type_analysis.loc[cluster_id, 'Movie'] if 'Movie' in type_analysis.columns else 0
    tv_shows = type_analysis.loc[cluster_id, 'TV Show'] if 'TV Show' in type_analysis.columns else 0
    total = movies + tv_shows
    
    if total > 0:
        movie_pct = movies / total
        if movie_pct >= 0.8:
            insights.append(f"🎬 Cluster {cluster_id} is primarily Movies ({movie_pct:.1%})")
        elif movie_pct <= 0.2:
            insights.append(f"📺 Cluster {cluster_id} is primarily TV Shows ({1-movie_pct:.1%})")

print("\n📋 INSIGHTS:")
for i, insight in enumerate(insights, 1):
    print(f"{i}. {insight}")

print(f"\n🚀 BUSINESS RECOMMENDATIONS:")
recommendations = [
    "🎯 Use clusters for personalized content recommendations",
    "📈 Optimize content acquisition based on cluster preferences", 
    "🎪 Create targeted marketing campaigns for each cluster",
    "📊 Analyze viewing patterns within clusters for content planning",
    "🌟 Develop cluster-specific user interface experiences",
    "📺 Plan content production based on successful cluster patterns",
    "🔍 Monitor cluster evolution over time for trend analysis"
]

for i, rec in enumerate(recommendations, 1):
    print(f"{i}. {rec}")

print(f"\n📊 CLUSTERING PERFORMANCE SUMMARY:")
print(f"   • Optimal Clusters: {optimal_k}")
print(f"   • Silhouette Score: {final_silhouette:.3f}")
print(f"   • Inertia: {final_inertia:.2f}")
print(f"   • Total Shows Analyzed: {len(df_processed)}")
print(f"   • Features Used: {len(available_features)}")

print(f"\n🎉 Analysis Complete! Netflix shows successfully clustered into {optimal_k} meaningful groups.")